# Connect to ClickHouse Database

In [3]:
!mkdir -p /home/jupyter/datasphere/project/.clickhouse && \
wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" \
     --output-document /home/jupyter/datasphere/project/.clickhouse/root.crt && \
chmod 0600 /home/jupyter/datasphere/project/.clickhouse/root.crt

--2024-09-12 12:40:21--  https://storage.yandexcloud.net/cloud-certs/CA.pem
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3579 (3.5K) [application/x-x509-ca-cert]
Saving to: ‘/home/jupyter/datasphere/project/.clickhouse/root.crt’

/home/jupyter/datas 100%[===================>]   3.50K  --.-KB/s    in 0s      

2024-09-12 12:40:22 (828 MB/s) - ‘/home/jupyter/datasphere/project/.clickhouse/root.crt’ saved [3579/3579]



In [4]:
# указываем параметры подключения
CA = '/home/jupyter/datasphere/project/.clickhouse/root.crt' # если вы не меняли путь при скачивании сертификата

VERIFY = True
SECURE = True
USER = 'admin'
PASSWORD = 'hakaton1'
HOST = "rc1d-uq593a42e02ikh2k.mdb.yandexcloud.net" # адрес Clickhouse
PORT = 8443

In [5]:
!pip install clickhouse_connect > /dev/null

In [6]:
# импортируем библиотеку
import clickhouse_connect

# выполняем запрос - в случае успешного выполнения будет показана текущая версия Clickhouse
with clickhouse_connect.get_client(
            host=HOST, port=PORT, username=USER,
            password=PASSWORD, secure=SECURE, verify=VERIFY, ca_cert=CA) as ch_client:
        print(ch_client.command('SELECT version()'))

24.3.10.33


# Генерация IAM токена

In [7]:
# Устанавливаем пакеты, необходимые для обращения к YandexGPT API
%pip uninstall jwt
%pip install PyJWT -U

In [8]:
import requests
import json
import time
import jwt
import os

In [9]:
# Замените <идентификатор_сервисного_аккаунта> на ваше значение
service_account_id = "aje286em9u4ibs8572di"

In [11]:
with open('/content/authorized_key.json') as f:
    authorized_key = json.load(f)

In [12]:
# Замените <идентификатор_ключа> на ваше значение
key_id = "ajeo9id7551hpq15fv3o"
private_key = authorized_key['private_key']

In [13]:
now = int(time.time())
payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'iat': now,
        'exp': now + 360}

# Формирование JWT
encoded_token = jwt.encode(
    payload,
    private_key,
    algorithm='PS256',
    headers={'kid': key_id})

url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
x = requests.post(url,  headers={'Content-Type': 'application/json'}, json = {'jwt': encoded_token}).json()
iam_token = x['iamToken']

# RAG Pipeline

In [14]:
!pip install -q langchain_community langchain yandexcloud > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [15]:
# llm
folder = 'b1g01stbeeo96j8kfaiv'
model_uri = f'gpt://{folder}/yandexgpt/latest'
model_uri

'gpt://b1g01stbeeo96j8kfaiv/yandexgpt/latest'

In [16]:
from langchain_community.embeddings.yandex import YandexGPTEmbeddings

# Cоздаем объект YandexGPTEmbeddings для построения векторов с помощью YandexGPT
embeddings = YandexGPTEmbeddings(iam_token=iam_token, folder_id=folder)

In [17]:
from langchain_community.vectorstores import Clickhouse, ClickhouseSettings

ch_config = ClickhouseSettings(host=HOST, port=PORT, username=USER, password=PASSWORD)

docsearch = Clickhouse(embedding=embeddings, config=ch_config, verify=VERIFY, ca_cert=CA)

In [85]:
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 9},
)

In [86]:
question = "Создай страницу c Sidebar слева, Header сверху страницы, далее ниже по горизонтале 9 шт. Input, справа от Input по горизонтале будет 2 шт. Button, далее ниже Tabs, далее ниже по вертикале 11 шт. Input, внизу страницы будет 2 шт. Button."

In [87]:
from langchain.prompts import ChatPromptTemplate

template = """
Ты ассистент, который разделяет входящий запрос на несколько подзапросов.
В основном входящий запрос будет требовать сгенерировать веб-интерфейс по описанию.
Разбей запрос на набор подзадач генерации отдельных высокоуровневых веб-компонент.
Каждая подзадача должна соответствовать одной единственной веб-компоненте.
Если в запросе указано название компоненты, то обязательно сохрани его.
Если в запросе указано относительное положение компненты, то обязательно сохрани его и продублируй название компоненты относительно которой она будет располагаться.
Сгенерируй несколько подзапросов для запроса: {question}.

Обязательно соблюдай правила форматирования ответа:
- Ничего кроме самих подзапросов писать не нужно.
- Один подзапрос - одно предложение.
- Подзапросы должны быть разделены символом перевода строки '\n'.
- Не должно быть 2 '\n' подряд!

Пример:
Запрос пользователя - создай кнопку, слевой от неё текст люблю щенят
Твой ответ:
1. Создай первый элемент кнопка 'Button'
2. Добавь слево (c помощью Grid) от первого элемента (кнопки) второй элемент input c тестом "люблю щенят"
"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [88]:
from langchain_community.llms import YandexGPT

# llm for prompt decomposition
prompt_decomposition_model = YandexGPT(iam_token=iam_token, model_uri=model_uri, tempreture=0.35)

In [89]:
from langchain_core.output_parsers import StrOutputParser

# Chain
generate_queries_decomposition = ( prompt_decomposition | prompt_decomposition_model | StrOutputParser() | (lambda x: x.replace('\n\n','\n')))

# Run
question = generate_queries_decomposition.invoke({"question": question})

In [90]:
question

'1. Создай компонент Sidebar слева от основного содержимого страницы.\n2. Добавь компонент Header в верхней части страницы.\n3. Размести 9 компонентов Input ниже по горизонтали.\n4. Расположи 2 компонента Button справа от каждого из Input по горизонтали.\n5. Добавь компонент Tabs ниже.\n6. Разместите 11 компонентов Input ниже Tabs по вертикали.\n7. Завершите страницу двумя компонентами Button внизу.'

In [91]:
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """
Ты - генератор интерфейсов по запросу пользователя на естественном языке.
Твой ответ должен содержать только код и не должен содержать описания или другого постороннего текста.
Ты обязан использовать дизайн компоненты системы НЛМК.
Учитывай, что твой вывод должен быть валидным React.js кодом, который можно запускать без ошибок.
Не забывай импортировать элементы. Обрати внимание, что некоторые элементы cостоят из 2 <> : <Button></>,  а некторые из 1 <Input />.
Не пише React.js в начале кода!!
Не ставь многоточие, чтоб пропустить элементы. Каждый элемент должен быть прописан (написано 9 кнопок - 9 элементов должны быть )
Будь очень внимателен к расположению элементов относительно друг друга. Все элементы должны быть видны.
Ответь на вопрос, используя следующие элементы.Если можно использовать элемент - обязательно используй:
{context}

Используй Grid для позиционирования элементов.
Используй его обязятельно, урежем зп а то.

Вопрос: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nТы - генератор интерфейсов по запросу пользователя на естественном языке.\nТвой ответ должен содержать только код и не должен содержать описания или другого постороннего текста.\nТы обязан использовать дизайн компоненты системы НЛМК.\nУчитывай, что твой вывод должен быть валидным React.js кодом, который можно запускать без ошибок.\nНе забывай импортировать элементы. Обрати внимание, что некоторые элементы cостоят из 2 <> : <Button></>,  а некторые из 1 <Input />.\nНе пише React.js в начале кода!!\nНе ставь многоточие, чтоб пропустить элементы. Каждый элемент должен быть прописан (написано 9 кнопок - 9 элементов должны быть )\nБудь очень внимателен к расположению элементов относительно друг друга. Все элементы должны быть видны.\nОтветь на вопрос, используя следующие элементы.Если можно использовать элемент - обязател

In [92]:
from langchain_community.llms import YandexGPT

llm = YandexGPT(iam_token=iam_token, model_uri=model_uri, tempreture=0.35)

In [93]:
# Chain
chain = prompt | llm

In [94]:
results = docsearch.similarity_search(query=question, k=10)
print(len(results))
for i, doc in enumerate(results):
  if i == 1: break
  print(f"* {doc.page_content} [{doc.metadata}]")

10
* NavigationPanel

Описание:
Создай панель навигации с логотипом и тремя ссылками: 'Главная', 'О нас', 'Контакты'.

Код:
import React, { useState } from "react";
import * as DS from "@nlmk/ds-2.0";

const NavigationPanel: React.FC = () => {
  const [currentPath, setCurrentPath] = useState("");

  return (
    <DS.Sidebar
      key="vertical"
      orientation="horizontal"
      allowFavorites
      currentPath={currentPath}
      onOpenUser={() => {
        /* Handle user profile opening */
      }}
    >
      <DS.Sidebar.Avatar imageSrc="logo.svg"></DS.Sidebar.Avatar>
      <DS.Sidebar.MenuItem
        path="home"
        label="Главная"
        position="top"
        icon="IconHomeOutlined24"
        onClick={() => setCurrentPath("home")}
      />
      <DS.Sidebar.MenuItem
        path="about"
        label="О нас"
        position="top"
        icon="IconInfoOutlined24"
        onClick={() => setCurrentPath("about")}
      />
      <DS.Sidebar.MenuItem
        path="contacts"
 

In [95]:
# Grid retrieval
grid_description = docsearch.similarity_search(query="Grid", k=1)[0]

In [96]:
print(grid_description)

page_content='Grid

Описание:
Компонент Grid представляет собой универсальный контейнер, используемый для позиционирования внутренних компонентов/элементов: горизонтальный или вертикальный. Он обладает различными пропсами, что делает Grid инструментом для создания структурированного и адаптивного интерфейса.
Горизонтальное выравнивание

Код:
import { Grid, Box } from '@nlmk/ds-2.0';

const App = () => (
    <Grid borderRadius="var(--4-border)">
      <Grid.Row borderRadius="var(--4-border)" background="var(--error-red-100)" justifyContent="center">
        <Grid.Column borderRadius="var(--4-border)" background="var(--primary-blue-400)" width="33.33%">
          <Box
            st={{ flex: '1' }}
            px="var(--8-space)"
            py="var(--16-space)"
            borderRadius="var(--4-border)"
            background="var(--primary-blue-400)"
          >
            Одна из двух колонок
          </Box>
        </Grid.Column>
        <Grid.Column borderRadius="var(--4-border)" 

In [97]:
results = docsearch.similarity_search(query=question, k=10)
# for doc in results: print(f"* {doc.page_content} [{doc.metadata}]")

In [98]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def add_grid_description(docs):
    docs.append(grid_description)
    return docs

def print_prompt(prompt):
    print(prompt.messages[0].content)
    return prompt

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


rag_chain = (
    {"context": retriever | add_grid_description | format_docs, "question": RunnablePassthrough()}
    | prompt
    | print_prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke(question)
print(response)


Ты - генератор интерфейсов по запросу пользователя на естественном языке.
Твой ответ должен содержать только код и не должен содержать описания или другого постороннего текста.
Ты обязан использовать дизайн компоненты системы НЛМК.
Учитывай, что твой вывод должен быть валидным React.js кодом, который можно запускать без ошибок.
Не забывай импортировать элементы. Обрати внимание, что некоторые элементы cостоят из 2 <> : <Button></>,  а некторые из 1 <Input />.
Не пише React.js в начале кода!!
Не ставь многоточие, чтоб пропустить элементы. Каждый элемент должен быть прописан (написано 9 кнопок - 9 элементов должны быть )
Будь очень внимателен к расположению элементов относительно друг друга. Все элементы должны быть видны.
Ответь на вопрос, используя следующие элементы.Если можно использовать элемент - обязательно используй:
NavigationPanel

Описание:
Создай панель навигации с логотипом и тремя ссылками: 'Главная', 'О нас', 'Контакты'.

Код:
import React, { useState } from "react";
impo